Make sure to do "gunzip -c file.json > file.json" to convert into a proper and usable format.

In [186]:
import ujson

In [187]:
def get_tweet_info(data_obj):
    try:
        name = data_obj[1]["result"][0]["user"]["name"]
    except:
        name = ''
    
    try:
        text = data_obj[1]["result"][0]["full_text"]
    except: 
        text = ''
    
    try:
        likes = data_obj[1]["result"][0]["user"]["favourites_count"]
    except:
        likes = 0
    
    try:
        hashtags_list = data_obj[1]["result"][0]["entities"]["hashtags"]
        hashtags = set()
        for obj in hashtags_list:
            hashtags.add(obj['text'])
    except:
        hashtags = set()
    
    try:
        date = data_obj[1]["result"][0]["created_at"]
    except:
        date = ''
        
    return [name, text, likes, hashtags, date]

In [188]:
def get_information(file):
    objects = list()
    count = 0
    for i, line in enumerate(file):
        tweet_data = ujson.loads(line)
        tweet = get_tweet_info(tweet_data).copy()
        if tweet[1] == '':
            pass
        else:
            objects.append(tweet)
            count += 1
    return objects

In [189]:
def all_hashtags(tweet_objects):
    hashtags = set()
    for obj in tweet_objects:
        hashtags.update(obj[3])
    return hashtags

In [190]:
def hashtag_freq(file):
    hash_dict = {}
    for i, line in enumerate(file):
        tweet_data = ujson.loads(line)
        try:
            hashtags_list = tweet_data[1]["result"][0]["entities"]["hashtags"]
            for obj in hashtags_list:
                if obj['text'] in hash_dict.keys():
                    hash_dict[obj['text']] += 1
                else:
                    hash_dict[obj['text']] = 1
        except:
            pass
    return hash_dict

In [191]:
def count_tweets(tweet_objects):
    count = 0
    for obj in tweet_objects:
        if obj[1] == '':
            pass
        else:
            count += 1
    return count

In [192]:
def count_users(tweet_objects):
    users = set()
    for obj in tweet_objects:
        if obj[0] == '':
            pass
        else:
            users.add(obj[0])
    return len(users)

In [193]:
def count_hashtags(tweet_objects):
    return len(all_hashtags(tweet_objects))

In [194]:
def count_tweets_without_hashtags(tweet_objects):
    count = 0
    for obj in tweet_objects:
        if obj[3] == set():
            count += 1
    return count

In [195]:
def date_range(tweet_objects):
    month = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    earliest = [3000, 12, 31]
    latest = [1900, 1, 1]
    for obj in tweet_objects:
        created_at = obj[4].split(" ")
        date = [int(created_at[5]), int(month[created_at[1]]), int(created_at[2])]
        
        #earlier
        if (earliest[0] > date[0]):
            earliest = date.copy()
        elif (earliest[0] == date[0]):
            if (earliest[1] > date[1]):
                earliest = date.copy()
            elif (earliest[1] == date[1]):
                if (earliest[2] > date[2]):
                    earliest = date.copy()
        
        #latest
        if (latest[0] < date[0]):
            latest = date.copy()
        elif (latest[0] == date[0]):
            if (latest[1] < date[1]):
                latest = date.copy()
            elif (latest[1] == date[1]):
                if (latest[2] < date[2]):
                    latest = date.copy()
    return [earliest, latest]

In [ ]:
file = open("/Users/sgupta/Desktop/MIT/Research/LSM/datasets/atlanta-tweets/data_3.json") 
tweet_objects = get_information(file)

#count
tweet_count = count_tweets(tweet_objects)
print('Tweets:', tweet_count)
user_count = count_users(tweet_objects)
print('Users:', user_count)
# hashtags_count = count_hashtags(tweet_objects)
hashtags = all_hashtags(tweet_objects)
print('Unique Hashtags:', len(hashtags))
no_hashtags_count = count_tweets_without_hashtags(tweet_objects)
print('Tweets without Hashtags:', no_hashtags_count)
range = date_range(tweet_objects)
early = str(range[0][1]) + '-' + str(range[0][2]) + '-' + str(range[0][0])
late = str(range[1][1]) + '-' + str(range[1][2]) + '-' + str(range[1][0])
print('Date Range:', early, 'to', late)

hashtags = all_hashtags(tweet_objects)
print('All Hashtags:')
print(hashtags)

In [ ]:
#hashtags
file = open("/Users/sgupta/Desktop/MIT/Research/LSM/datasets/atlanta-tweets/data_3.json") 
frequency = hashtag_freq(file)
print("Frequency of Hashtags:")
print(frequency)

In [ ]:
file = open("/Users/sgupta/Desktop/MIT/Research/LSM/datasets/atlanta-tweets/data_3.json") 
for i, line in enumerate(file):
    tweet_data = ujson.loads(line)
    json_formatted_str = ujson.dumps(tweet_data, indent=4)
    if i == 2:
        print(json_formatted_str)
        break